<h1>SMS SPAM DETECTION MODEL</h1>

<p>In this project we are trying to make a model to classify a message as spam or ham</p>
<p>In that model we will train and test data using different machine learning algorithms and find out which algorithm works best in this dataset</p>

In [2]:
##Importing all the required libraries
#%pip install ipykernel
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.naive_bayes import MultinomialNB,GaussianNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
import seaborn as sns

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score

ModuleNotFoundError: No module named 'pandas'

In [ ]:
#Loading dataset
data = pd.read_csv("spam.csv",encoding='latin-1')
data.head()

: 

In [ ]:
#Droping unwanted columns
data = data.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4'],axis=1)

#Naming the columns
data = data.rename(columns = {'v1':'label','v2':'message'})


In [ ]:
data.describe()

In [ ]:
data.groupby('label').describe()

In [ ]:
#Calculating the lenght of all the messages
data['lenght']=data['message'].apply(len)
data.head()


In [ ]:
data.describe()

In [ ]:
%matplotlib inline

#Plotting the lenght of ham and spam messages
data.hist(by='label',column='lenght',bins=30,figsize=[15,5]) 

<p>From this plot we can say that spam messages are much more long than ham messages on an average</p>


In [ ]:
#Converting our labels to numeric labels
# ham = 0 and spam = 1
data['label_num']=data.label.map({'ham':0,'spam':1})
data.head()

In [ ]:
#loading our features and target to train
x = data.message
y = data.label_num


In [ ]:
#Now we have 5572 labels for 5572 features
#spliting data into 75% test and 25% into train which is by default
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=11)


In [ ]:
vect = CountVectorizer()
#converting features into numeric vector
X_train = vect.fit_transform(x_train)
#converting target into numeric vector
X_test = vect.transform(x_test)





In [ ]:
#Loading all classifier
svc = SVC(kernel = 'linear')
mnb = MultinomialNB(alpha =0.2)
gnb  = GaussianNB()
lr = LogisticRegression(solver='liblinear', penalty='l1')
rfc = RandomForestClassifier(n_estimators=100,random_state=11)
abc = AdaBoostClassifier(n_estimators =100,random_state=11)

In [ ]:
#defining functions for training and testing data

def training(clf,x_train,Y_train):
    clf.fit(x_train,Y_train)
    
#function for predicting labels

def predict(clf,X_test):
    return clf.predict(X_test)

In [ ]:
#defining a dictionary of classifier
classifier={'SVM': svc , 'MultinomialNB': mnb,'GaussianNB': gnb,'logistic': lr,'RandomForest': rfc,'Adaboost': abc}

In [ ]:
#predict and storing score of each classifier
score = []
for n,c in classifier.items():
    training(c,X_train.toarray(),y_train)
    pred = predict(c,X_test.toarray())
    score.append((n,[accuracy_score(y_test,pred,normalize=True)]))
    

In [ ]:
score_df=pd.DataFrame.from_items(score,orient='index',columns=['scores'])
#Adding accuracy column
score_df['Accuracy (%)']=score_df['scores']*100
score_df

<p>We can see svm has the best accuracy of 0.989 followed by multinomial naiye bayes which have accuracy of 0.988</p>

<p>Also notice that Gaussian NB work very poorly</p>

In [ ]:
##Find out time taken by each classifier
##remove stopwords and train again
##use GridSearchCV to tune algorithms

In [ ]:
 #Implementing K-Nearest Neighbors (KNN)
from sklearn.neighbors import KNeighborsClassifier

# Initialize KNN with k=5 (can be tuned)
knn = KNeighborsClassifier(n_neighbors=5)

# Train the model
knn.fit(X_train, y_train)

# Predict on test set
y_pred_knn = knn.predict(X_test)

# Evaluate model
knn_accuracy = accuracy_score(y_test, y_pred_knn)
print(f"KNN Accuracy: {knn_accuracy:.4f}")

In [ ]:
#conda create --name tf_env python=3.10


In [ ]:
#%pip install tensorflow
#import tensorflow as tf

%pip show tensorflow

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, LSTM, Dense, SpatialDropout1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Tokenization and Sequence Padding
max_words = 5000  # Vocabulary size
max_len = 100  # Max sequence length

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)

# Convert labels to numerical format
y_train_bin = (y_train == 'spam').astype(int)
y_test_bin = (y_test == 'spam').astype(int)

# Build RNN Model
rnn_model = Sequential([
    Embedding(max_words, 32, input_length=max_len),
    SimpleRNN(32, return_sequences=True),
    SimpleRNN(16),
    Dense(1, activation='sigmoid')
])

rnn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
rnn_model.fit(X_train_pad, y_train_bin, epochs=5, batch_size=32, validation_data=(X_test_pad, y_test_bin))

# Evaluate RNN Model
rnn_accuracy = rnn_model.evaluate(X_test_pad, y_test_bin, verbose=0)[1]
print(f"RNN Accuracy: {rnn_accuracy:.4f}")

# Build LSTM Model
lstm_model = Sequential([
    Embedding(max_words, 64, input_length=max_len),
    SpatialDropout1D(0.3),
    LSTM(64, dropout=0.3, recurrent_dropout=0.3),
    Dense(1, activation='sigmoid')
])

lstm_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
lstm_model.fit(X_train_pad, y_train_bin, epochs=5, batch_size=32, validation_data=(X_test_pad, y_test_bin))

# Evaluate LSTM Model
lstm_accuracy = lstm_model.evaluate(X_test_pad, y_test_bin, verbose=0)[1]
print(f"LSTM Accuracy: {lstm_accuracy:.4f}")
